In [ ]:
from flask import Flask, render_template, Response
import datetime
import cv2
import numpy as np
import time
import sys
import boto3

import boto3

service_name = 's3'
endpoint_url = 'https://kr.object.ncloudstorage.com'
region_name = 'kr-standard'
access_key = 'fMwo4ZlCS60SGzwjoxb1'
secret_key = 'kb0CgwTfxQi6uhvdfaT1yxgysOsCZnMbD6xTdC8G'

object_list = []

storage_link = 'https://kr.object.ncloudstorage.com/'
bucket_name = ''


if __name__ == "__main__":
    s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                      aws_secret_access_key=secret_key)

    response = s3.list_buckets()
    
    #bucket name print
    for bucket in response.get('Buckets', []):
        #print(bucket.get('Name'))
        bucket_name = bucket.get('Name')

    # list all in the bucket
    max_keys = 300
    response = s3.list_objects(Bucket=bucket_name, MaxKeys=max_keys)

    print('list all in the bucket')

    while True:
        print('IsTruncated=%r' % response.get('IsTruncated'))
        print('Marker=%s' % response.get('Marker'))
        print('NextMarker=%s' % response.get('NextMarker'))

        print('Object List')
        for content in response.get('Contents'):
            if content.get('Key').startswith("video/") and content.get('Key').endswith(".mp4"):
                print(' Name=%s' % (content.get('Key')))
                object_list.append(content.get('Key'))

        if response.get('IsTruncated'):
            response = s3.list_objects(Bucket=bucket_name, MaxKeys=max_keys,
                                       Marker=response.get('NextMarker'))
        else:
            break

print('스토리지')
print(storage_link)
print('버킷')
print(bucket_name)
print('영상들')
print(object_list)
print()
print('합쳐서')
print(storage_link+bucket_name+'/'+object_list[0])


In [ ]:
faceCascade = cv2.CascadeClassifier("../face/xml/haarcascade_frontalface_default.xml")

app = Flask(__name__)
@app.route('/')
def index():
    """Video streaming home page."""
    now = datetime.datetime.now()
    timeString = now.strftime("%Y-%m-%d %H:%M")
    templateData = {
            'title':'Image Streaming',
            'time': timeString
            }
    return render_template('index.html', **templateData)
def gen_frames():
    #camera = cv2.VideoCapture(0)
    camera = cv2.VideoCapture(storage_link + bucket_name + '/' + object_list[0])
    time.sleep(0.2)
    lastTime = time.time()*1000.0

    while True:
        ret, image = camera.read()
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
       
        faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=6)
        delt = time.time()*1000.0-lastTime
        s = str(int(delt))
        #print (delt," Found {0} faces!".format(len(faces)) )
        lastTime = time.time()*1000.0
        
        # Draw a rectangle around the faces
        for (x, y, w, h) in faces:
            #cv2.circle(image, (int(x+w/2), int(y+h/2)), int((w+h)/3), (255, 255, 255), 3)
            cv2.rectangle(image, (x,y), (x+w, y+h), (255,255,255), thickness=2)
            #cv2.putText(image,info,(x,y-15),0, 0.5, (0, 255, 0), 1)
        cv2.putText(image, s, (10, 25),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        now = datetime.datetime.now()
        timeString = now.strftime("%Y-%m-%d %H:%M")
        cv2.putText(image, timeString, (10, 45),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        cv2.imshow("Frame", image)
        key = cv2.waitKey(1) & 0xFF
     # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
   
        ret, buffer = cv2.imencode('.jpg', image)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
       
 
@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0')       